#**StyleGAN Project**
Authors:

Brayam Castillo, Gabriel Baker, Filipe Lauar, Vinícius Imaizumi

Note:

Most of the code was about using the github repositories, without much "original" code by us.

Also, we had really a lot of problems trying to use Caffe in Colab. We sucessed only when installing locally, thus the code using the FaceAttribute-FAN isn't here. We only used the network to predict images, without changing its code.

The function used to plot the graphs from the output of FaceAttribute-FAN was defined in the last cell of this Colab, but we used it only locally in the computer with Caffe.

## StyleGAN

In [1]:
%tensorflow_version 1.x # required to use StyleGAN
import tensorflow
print(tensorflow.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # required to use StyleGAN`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [2]:
! git clone https://github.com/NVlabs/stylegan

Cloning into 'stylegan'...
remote: Enumerating objects: 86, done.
remote: Total 86 (delta 0), reused 0 (delta 0), pack-reused 86
Unpacking objects: 100% (86/86), done.


In [3]:
import os

In [4]:
os.chdir("stylegan")  # changing working dir to do the imports properly

In [5]:
import pickle
import numpy as np
import PIL.Image
import dnnlib as dnnlib
import dnnlib.tflib as tflib
import config as config

In [6]:
# Loads the model StyleGAN FFHQ 1024x1024 from personal google drive link

# Initialize TensorFlow.
tflib.init_tf()
# Load pre-trained network.
url = 'https://drive.google.com/uc?id=1_zKCCV3R7KSDLlgQpz1PbMGTUy4rY97v' # karras2019stylegan-ffhq-1024x1024.pkl
with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
    _G, _D, Gs = pickle.load(f)











Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
# Pick latent vector.
rnd = np.random.RandomState(1000)
total_imgs = 100

all_latents = np.empty((total_imgs, Gs.input_shape[1])) # save all images 
# OOM error if we try 100 images at the same time
for j in range(total_imgs//50):
    latents = rnd.randn(50, Gs.input_shape[1])
    all_latents[j*50:j*50+50] = latents.copy()

    # Generate image.
    fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    images = Gs.run(latents, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

    # Save image.
    os.makedirs(config.result_dir, exist_ok=True)
    for i, image in enumerate(images):
        png_filename = os.path.join(config.result_dir, f'example_{i+50*j}.png')
        PIL.Image.fromarray(images[i], 'RGB').save(png_filename)


In [8]:
# Saving all images in the latent space W
all_src_dlatents = np.empty((total_imgs, 18, Gs.input_shape[1]))
for j in range(total_imgs//50):
    src_dlatents = Gs.components.mapping.run(all_latents[j*50:j*50+50], None)
    all_src_dlatents[j*50:j*50+50] = src_dlatents.copy()

In [9]:
# Block to do the weighted average between two images

weights = np.linspace(0, 1, 10)
# We chose manually the images at index 0 and index 11, as we can see below
latent_mixed = [weights[i]*all_latents[0] + (1-weights[i])*all_latents[11] for i in range(len(weights))]
latent_mixed = np.array(latent_mixed)
img_mixed = Gs.run(latent_mixed, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)

# Saving images in the folder ./stylegan/linvar
os.makedirs('./linvar', exist_ok=True)
for i, image in enumerate(img_mixed):
    png_filename = os.path.join('./linvar', f'{round(100*weights[i])} {round(100-100*weights[i])}.png')
    PIL.Image.fromarray(img_mixed[i], 'RGB').save(png_filename)

In [10]:
# Changing manually a coordinate in the latent space W to get an artifact
i = 46
test_image = all_latents[0].copy()
test_image[46] = 20#-np.cos(i)*2*i
test_image = np.array([test_image])
!mkdir ./tests
png_filename = os.path.join('./tests/', f'test{i}.png')
PIL.Image.fromarray(Gs.run(test_image, None, truncation_psi=0.7, randomize_noise=True, output_transform=fmt)[0], 'RGB').save(png_filename)

In [ ]:
# Saving others images to use in the InterfaceGAN
# Tried to chose images with a good variability of characteristics

os.makedirs('./latents', exist_ok=True)
imgs_chosen = [0, 1, 8, 10, 11, 13, 14, 32, 55, 58, 68, 95, 97]
np.save(f'./latents/seed1000_imgs.npy', all_src_dlatents[imgs_chosen][:, 0])

In [ ]:
# We go back to root dir
os.chdir("..")
!ls

sample_data  stylegan


## InterfaceGAN

We always ran out of memory in this part if we didn't reset the environment once or more times. To use this, after saving the images to be used in InterfaceGAN one must reset the environment until we don't run OOM when using InterfaceGAN.

In [ ]:
! git clone https://github.com/genforce/interfacegan

Cloning into 'interfacegan'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 252 (delta 0), reused 0 (delta 0), pack-reused 247
Receiving objects: 100% (252/252), 11.39 MiB | 40.37 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1_zKCCV3R7KSDLlgQpz1PbMGTUy4rY97v',
                                    dest_path='./interfacegan/models/pretrain/karras2019stylegan-ffhq-1024x1024.pkl',
                                    unzip=True)

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1_zKCCV3R7KSDLlgQpz1PbMGTUy4rY97v" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
# If you run two cell aboves and fail, you must delete all folder created to run again
# Which is pretty sad when we need to restart the environment multiple times

!rm -r ./interfacegan/results/stylegan_ffhq_age
!rm -r ./interfacegan/results/stylegan_ffhq_pose
!rm -r ./interfacegan/results/stylegan_ffhq_smile
!rm -r ./interfacegan/results/stylegan_ffhq_gender
!rm -r ./interfacegan/results/stylegan_ffhq_eyeglasses

rm: cannot remove './interfacegan/results/stylegan_ffhq_age': No such file or directory
rm: cannot remove './interfacegan/results/stylegan_ffhq_smile': No such file or directory
rm: cannot remove './interfacegan/results/stylegan_ffhq_gender': No such file or directory
rm: cannot remove './interfacegan/results/stylegan_ffhq_eyeglasses': No such file or directory


In [ ]:
# Command line to run interfacegan in the age boundary

%tensorflow_version 1.x

!python ./interfacegan/edit.py -s W \
    -m stylegan_ffhq \
    -b ./interfacegan/boundaries/stylegan_ffhq_age_w_boundary.npy \
    -i "./stylegan/latents/seed1000_imgs.npy" \
    --start_distance -4 \
    --end_distance 4 \
    -o ./interfacegan/results/stylegan_ffhq_age \
    --steps 9

[2021-06-21 09:39:10,899][INFO] Initializing generator.
[2021-06-21 09:39:11,126][INFO] Loading pytorch model from `interfacegan/models/pretrain/stylegan_ffhq.pth`.
[2021-06-21 09:39:11,354][INFO] Successfully loaded!
[2021-06-21 09:39:11,355][INFO]   `lod` of the loaded model is 0.0.
[2021-06-21 09:39:14,658][INFO] Preparing boundary.
[2021-06-21 09:39:14,662][INFO] Preparing latent codes.
[2021-06-21 09:39:14,662][INFO]   Load latent codes from `./stylegan/latents/seed1000_imgs.npy`.
[2021-06-21 09:39:14,662][INFO] Editing 13 samples.
[2021-06-21 09:39:24,218][INFO] Successfully edited 13 samples.


In [ ]:
# Pose boundary, ..., etc

%tensorflow_version 1.x

!python ./interfacegan/edit.py -s W \
    -m stylegan_ffhq \
    -b ./interfacegan/boundaries/stylegan_ffhq_pose_w_boundary.npy \
    -i "./stylegan/latents/seed1000_imgs.npy" \
    --start_distance -4 \
    --end_distance 4 \
    -o ./interfacegan/results/stylegan_ffhq_pose \
    --steps 9

[2021-06-21 09:39:26,524][INFO] Initializing generator.
[2021-06-21 09:39:26,747][INFO] Loading pytorch model from `interfacegan/models/pretrain/stylegan_ffhq.pth`.
[2021-06-21 09:39:26,999][INFO] Successfully loaded!
[2021-06-21 09:39:27,000][INFO]   `lod` of the loaded model is 0.0.
[2021-06-21 09:39:30,266][INFO] Preparing boundary.
[2021-06-21 09:39:30,270][INFO] Preparing latent codes.
[2021-06-21 09:39:30,270][INFO]   Load latent codes from `./stylegan/latents/seed1000_imgs.npy`.
[2021-06-21 09:39:30,271][INFO] Editing 13 samples.
[2021-06-21 09:39:39,861][INFO] Successfully edited 13 samples.


In [ ]:
# Smile

%tensorflow_version 1.x

!python ./interfacegan/edit.py -s W \
    -m stylegan_ffhq \
    -b ./interfacegan/boundaries/stylegan_ffhq_smile_w_boundary.npy \
    -i "./stylegan/latents/seed1000_imgs.npy" \
    --start_distance -4 \
    --end_distance 4 \
    -o ./interfacegan/results/stylegan_ffhq_smile \
    --steps 9

[2021-06-21 09:39:41,417][INFO] Initializing generator.
[2021-06-21 09:39:41,635][INFO] Loading pytorch model from `interfacegan/models/pretrain/stylegan_ffhq.pth`.
[2021-06-21 09:39:41,856][INFO] Successfully loaded!
[2021-06-21 09:39:41,857][INFO]   `lod` of the loaded model is 0.0.
[2021-06-21 09:39:45,119][INFO] Preparing boundary.
[2021-06-21 09:39:45,123][INFO] Preparing latent codes.
[2021-06-21 09:39:45,123][INFO]   Load latent codes from `./stylegan/latents/seed1000_imgs.npy`.
[2021-06-21 09:39:45,123][INFO] Editing 13 samples.
[2021-06-21 09:39:54,697][INFO] Successfully edited 13 samples.


In [ ]:
# Gender

%tensorflow_version 1.x

!python ./interfacegan/edit.py -s W \
    -m stylegan_ffhq \
    -b ./interfacegan/boundaries/stylegan_ffhq_gender_w_boundary.npy \
    -i "./stylegan/latents/seed1000_imgs.npy" \
    --start_distance -4 \
    --end_distance 4 \
    -o ./interfacegan/results/stylegan_ffhq_gender \
    --steps 9

[2021-06-21 09:39:56,211][INFO] Initializing generator.
[2021-06-21 09:39:56,435][INFO] Loading pytorch model from `interfacegan/models/pretrain/stylegan_ffhq.pth`.
[2021-06-21 09:39:56,682][INFO] Successfully loaded!
[2021-06-21 09:39:56,682][INFO]   `lod` of the loaded model is 0.0.
[2021-06-21 09:39:59,954][INFO] Preparing boundary.
[2021-06-21 09:39:59,958][INFO] Preparing latent codes.
[2021-06-21 09:39:59,958][INFO]   Load latent codes from `./stylegan/latents/seed1000_imgs.npy`.
[2021-06-21 09:39:59,959][INFO] Editing 13 samples.
[2021-06-21 09:40:09,573][INFO] Successfully edited 13 samples.


In [ ]:
# Eyeglasses

%tensorflow_version 1.x

!python ./interfacegan/edit.py -s W \
    -m stylegan_ffhq \
    -b ./interfacegan/boundaries/stylegan_ffhq_eyeglasses_w_boundary.npy \
    -i "./stylegan/latents/seed1000_imgs.npy" \
    --start_distance -4 \
    --end_distance 4 \
    -o ./interfacegan/results/stylegan_ffhq_eyeglasses \
    --steps 9

[2021-06-21 09:40:11,148][INFO] Initializing generator.
[2021-06-21 09:40:11,368][INFO] Loading pytorch model from `interfacegan/models/pretrain/stylegan_ffhq.pth`.
[2021-06-21 09:40:11,604][INFO] Successfully loaded!
[2021-06-21 09:40:11,605][INFO]   `lod` of the loaded model is 0.0.
[2021-06-21 09:40:14,871][INFO] Preparing boundary.
[2021-06-21 09:40:14,875][INFO] Preparing latent codes.
[2021-06-21 09:40:14,875][INFO]   Load latent codes from `./stylegan/latents/seed1000_imgs.npy`.
[2021-06-21 09:40:14,876][INFO] Editing 13 samples.
[2021-06-21 09:40:24,492][INFO] Successfully edited 13 samples.


In [ ]:
# Zipping and downloading results

!zip -r ./results.zip ./interfacegan/results
from google.colab import files
files.download("./results.zip")

!zip -r ./linvar.zip ./linvar
from google.colab import files
files.download("./linvar.zip")

## Image inversion (from real image to latent space W)

The blocks of code below are almost the same as found in the repository https://github.com/genforce/idinvert_pytorch

In [ ]:
import os

os.chdir('/content')
CODE_DIR = 'idinvert'
if not os.path.exists(CODE_DIR):
  !git clone https://github.com/genforce/idinvert_pytorch.git $CODE_DIR
os.chdir(f'./{CODE_DIR}')
MODEL_DIR = os.path.join('models', 'pretrain')
os.makedirs(MODEL_DIR, exist_ok=True)
!wget https://mycuhk-my.sharepoint.com/:u:/g/personal/1155082926_link_cuhk_edu_hk/EXqix_JIEgtLl1FXI4uCkr8B5GPaiJyiLXL6cFbdcIKqEA?e=WYesel\&download\=1 -O $MODEL_DIR/styleganinv_ffhq256_encoder.pth  --quiet
!wget https://mycuhk-my.sharepoint.com/:u:/g/personal/1155082926_link_cuhk_edu_hk/EbuzMQ3ZLl1AqvKJzeeBq7IBoQD-C1LfMIC8USlmOMPt3Q?e=CMXn8W\&download\=1 -O $MODEL_DIR/styleganinv_ffhq256_generator.pth  --quiet
!wget https://mycuhk-my.sharepoint.com/:u:/g/personal/1155082926_link_cuhk_edu_hk/EQJUz9DInbxEnp0aomkGGzAB5b3ZZbtsOA-TXct9E4ONqA?e=smtO0T\&download\=1 -O $MODEL_DIR/vgg16.pth  --quiet
!nvidia-smi

In [ ]:
# python 3.6
"""Demo."""
import os
import sys
import io
import bz2
import requests
import dlib
import numpy as np
from PIL import Image
import IPython.display
import scipy.ndimage
from google.colab import files
from google.colab import output
from utils.editor import manipulate
from utils.inverter import StyleGANInverter
from models.helper import build_generator


LANDMARK_MODEL_NAME = 'shape_predictor_68_face_landmarks.dat'
LANDMARK_MODEL_PATH = os.path.join(MODEL_DIR, LANDMARK_MODEL_NAME)
LANDMARK_MODEL_URL = f'http://dlib.net/files/{LANDMARK_MODEL_NAME}.bz2'
model_name = 'styleganinv_ffhq256'
pre = 'examples'
inverted_code_dir = 'inverted_codes'
os.makedirs(inverted_code_dir, exist_ok=True)

class FaceLandmarkDetector(object):
  """Class of face landmark detector."""

  def _init_(self, align_size=256, enable_padding=True):
    """Initializes face detector and landmark detector.

  Args:
    align_size: Size of the aligned face if performing face alignment.
    (default: 1024)
    enable_padding: Whether to enable padding for face alignment (default:
    True)
  """
    # Download models if needed.
    if not os.path.exists(LANDMARK_MODEL_PATH):
      data = requests.get(LANDMARK_MODEL_URL)
      data_decompressed = bz2.decompress(data.content)
      with open(LANDMARK_MODEL_PATH, 'wb') as f:
        f.write(data_decompressed)

    self.face_detector = dlib.get_frontal_face_detector()
    self.landmark_detector = dlib.shape_predictor(LANDMARK_MODEL_PATH)
    self.align_size = align_size
    self.enable_padding = enable_padding

  def detect(self, image_path):
    """Detects landmarks from the given image.

  This function will first perform face detection on the input image. All
  detected results will be grouped into a list. If no face is detected, an
  empty list will be returned.

  For each element in the list, it is a dictionary consisting of `image_path`,
  `bbox` and `landmarks`. `image_path` is the path to the input image. `bbox`
  is the 4-element bounding box with order (left, top, right, bottom), and
  `landmarks` is a list of 68 (x, y) points.

  Args:
    image_path: Path to the image to detect landmarks from.

  Returns:
    A list of dictionaries, each of which is the detection results of a
    particular face.
  """
    results = []

    # image_ = np.array(image)
    images = dlib.load_rgb_image(image_path)
    # Face detection (1 means to upsample the image for 1 time.)
    bboxes = self.face_detector(images, 1)
    # Landmark detection
    for bbox in bboxes:
      landmarks = []
      for point in self.landmark_detector(images, bbox).parts():
        landmarks.append((point.x, point.y))
      results.append({
          'image_path': image_path,
          'bbox': (bbox.left(), bbox.top(), bbox.right(), bbox.bottom()),
          'landmarks': landmarks,
      })
    return results

  def align(self, face_info):
    """Aligns face based on landmark detection.

  The face alignment process is borrowed from
  https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py,
  which only supports aligning faces to square size.

  Args:
    face_info: Face information, which is the element of the list returned by
    `self.detect()`.

  Returns:
    A `np.ndarray`, containing the aligned result. It is with `RGB` channel
    order.
  """
    img = Image.open(face_info['image_path'])

    landmarks = np.array(face_info['landmarks'])
    eye_left = np.mean(landmarks[36: 42], axis=0)
    eye_right = np.mean(landmarks[42: 48], axis=0)
    eye_middle = (eye_left + eye_right) / 2
    eye_to_eye = eye_right - eye_left
    mouth_middle = (landmarks[48] + landmarks[54]) / 2
    eye_to_mouth = mouth_middle - eye_middle

    # Choose oriented crop rectangle.
    x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
    x /= np.hypot(*x)
    x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
    y = np.flipud(x) * [-1, 1]
    c = eye_middle + eye_to_mouth * 0.1
    quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
    qsize = np.hypot(*x) * 2

    # Shrink.
    shrink = int(np.floor(qsize / self.align_size * 0.5))
    if shrink > 1:
      rsize = (int(np.rint(float(img.size[0]) / shrink)),
               int(np.rint(float(img.size[1]) / shrink)))
      img = img.resize(rsize, Image.ANTIALIAS)
      quad /= shrink
      qsize /= shrink

    # Crop.
    border = max(int(np.rint(qsize * 0.1)), 3)
    crop = (int(np.floor(min(quad[:, 0]))), int(np.floor(min(quad[:, 1]))),
            int(np.ceil(max(quad[:, 0]))), int(np.ceil(max(quad[:, 1]))))
    crop = (max(crop[0] - border, 0),
            max(crop[1] - border, 0),
            min(crop[2] + border, img.size[0]),
            min(crop[3] + border, img.size[1]))
    if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
      img = img.crop(crop)
      quad -= crop[0:2]

    # Pad.
    pad = (int(np.floor(min(quad[:, 0]))), int(np.floor(min(quad[:, 1]))),
           int(np.ceil(max(quad[:, 0]))), int(np.ceil(max(quad[:, 1]))))
    pad = (max(-pad[0] + border, 0),
           max(-pad[1] + border, 0),
           max(pad[2] - img.size[0] + border, 0),
           max(pad[3] - img.size[1] + border, 0))
    if self.enable_padding and max(pad) > border - 4:
      pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
      img = np.pad(np.float32(img),
                   ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)),
                   'reflect')
      h, w, _ = img.shape
      y, x, _ = np.ogrid[:h, :w, :1]
      mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0],
                                         np.float32(w - 1 - x) / pad[2]),
                        1.0 - np.minimum(np.float32(y) / pad[1],
                                         np.float32(h - 1 - y) / pad[3]))
      blur = qsize * 0.02
      blurred_image = scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img
      img += blurred_image * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
      img += (np.median(img, axis=(0, 1)) - img) * np.clip(mask, 0.0, 1.0)
      img = Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
      quad += pad[:2]

    # Transform.
    img = img.transform((self.align_size * 4, self.align_size * 4), Image.QUAD,
                        (quad + 0.5).flatten(), Image.BILINEAR)
    img = img.resize((self.align_size, self.align_size), Image.ANTIALIAS)

    return np.array(img)


def align_face(image_path, align_size=256):
  """Aligns a given face."""
  model = FaceLandmarkDetector(align_size)
  face_infos = model.detect(image_path)
  face_infos = face_infos[0]
  img = model.align(face_infos)
  return img


def build_inverter(model_name, iteration=100, regularization_loss_weight=2):
  """Builds inverter"""
  inverter = StyleGANInverter(
      model_name,
      learning_rate=0.01,
      iteration=iteration,
      reconstruction_loss_weight=1.0,
      perceptual_loss_weight=5e-5,
      regularization_loss_weight=regularization_loss_weight)
  return inverter


def get_generator(model_name):
  """Gets model by name"""
  return build_generator(model_name)


def align(inverter, image_path):
  """Aligns an unloaded image."""
  aligned_image = align_face(image_path,
                             align_size=inverter.G.resolution)
  return aligned_image


def invert(inverter, image):
  """Inverts an image."""
  latent_code, reconstruction = inverter.easy_invert(image, num_viz=1)
  return latent_code, reconstruction


def diffuse(inverter, target, context, left, top, width, height):
  """Diffuses a target image to a context image."""
  center_x = left + width // 2
  center_y = top + height // 2
  _, diffusion = inverter.easy_diffuse(target=target,
                                       context=context,
                                       center_x=center_x,
                                       center_y=center_y,
                                       crop_x=width,
                                       crop_y=height,
                                       num_viz=1)
  return diffusion


def load_image(path):
  """Loads an image from disk.

  NOTE: This function will always return an image with `RGB` channel order for
  color image and pixel range [0, 255].

  Args:
    path: Path to load the image from.

  Returns:
    An image with dtype `np.ndarray` or `None` if input `path` does not exist.
  """
  if not os.path.isfile(path):
    return None

  image = Image.open(path)
  return image

def imshow(images, col, viz_size=256):
  """Shows images in one figure."""
  num, height, width, channels = images.shape
  assert num % col == 0
  row = num // col

  fused_image = np.zeros((viz_size * row, viz_size * col, channels), dtype=np.uint8)

  for idx, image in enumerate(images):
    i, j = divmod(idx, col)
    y = i * viz_size
    x = j * viz_size
    if height != viz_size or width != viz_size:
      image = cv2.resize(image, (viz_size, viz_size))
    fused_image[y:y + viz_size, x:x + viz_size] = image

  fused_image = np.asarray(fused_image, dtype=np.uint8)
  data = io.BytesIO()
  if channels == 4:
    Image.fromarray(fused_image).save(data, 'png')
  elif channels == 3:
    Image.fromarray(fused_image).save(data, 'jpeg')
  else:
    raise ValueError('Image channel error')
  im_data = data.getvalue()
  disp = IPython.display.display(IPython.display.Image(im_data))
  return disp

In [ ]:
print('Building inverter')
inverter = build_inverter(model_name=model_name)
print('Building generator')
generator = get_generator(model_name)
output.clear()
print('Please upload the image you want to manipulate or \
use the default images by clicking `Cancel upload` button.')
uploaded = files.upload()
if uploaded:
  image_name = list(uploaded.keys())[0]
  mani_image = align(inverter, image_name)
  if mani_image.shape[2] == 4:
    mani_image = mani_image[:, :, :3]
  os.remove(image_name)
else:
  image_name = '000006.png'
  im_name = os.path.join(pre, image_name)
  mani_image = align(inverter, im_name)
print('Image ready, starting inversion!!!')
sys.stdout.flush()

latent_code_path = os.path.join(inverted_code_dir, 
                                image_name.split('.')[0] + '.npy')
if not os.path.exists(latent_code_path):
  latent_code, _ = invert(inverter, mani_image)
  np.save(latent_code_path, latent_code)
else:
  print('code already exists, skip inversion!!!')
  latent_code = np.load(latent_code_path)

ATTRS = ['age', 'eyeglasses', 'gender', 'pose', 'expression']
boundaries = {}
for attr in ATTRS:
  boundary_path = os.path.join('./boundaries', 
                               'stylegan_ffhq256', attr + '.npy')
  boundary_file = np.load(boundary_path, allow_pickle=True)[()]
  boundary = boundary_file['boundary']
  manipulate_layers = boundary_file['meta_data']['manipulate_layers']
  boundaries[attr] = []
  boundaries[attr].append(boundary)
  boundaries[attr].append(manipulate_layers)
print()
print('Image inversion completed, please use the next block to manipulate!!!')

In [ ]:
#@title { display-mode: "form", run: "auto" }

age = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
eyeglasses = 0 #@param {type:"slider", min:-2.9, max:3.0, step:0.1}
gender = -2.4 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
pose = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}
expression = 0 #@param {type:"slider", min:-3.0, max:3.0, step:0.1}


new_codes = latent_code.copy()
for i, attr_name in enumerate(ATTRS):
  manipulate_layers = boundaries[attr_name][1]
  new_codes[:, manipulate_layers, :] += boundaries[attr_name][0][:, manipulate_layers, :] * eval(attr_name)

new_images = generator.easy_synthesize(new_codes, **{'latent_space_type': 'wp'})['image']
showed_images = np.concatenate([mani_image[np.newaxis], new_images], axis=0)
imshow(showed_images, col=showed_images.shape[0])

### Interpolation

In [ ]:
print('Building inverter')
inverter = build_inverter(model_name=model_name)
print('Building generator')
generator = get_generator(model_name)
output.clear()
def linear_interpolate(src_code, dst_code, step=5):
  """Interpolates two latent codes linearlly.
  Args:
    src_code: Source code, with shape [1, latent_space_dim].
    dst_code: Target code, with shape [1, latent_space_dim].
    step: Number of interploation steps. (default: 5)
  Returns:
    Interpolated code, with shape [step, latent_space_dim].
  """
  assert (len(src_code.shape) == 2 and len(dst_code.shape) == 2 and
          src_code.shape[0] == 1 and dst_code.shape[0] == 1 and
          src_code.shape[1] == dst_code.shape[1])

  linspace = np.linspace(0.0, 1.0, step)[:, np.newaxis].astype(np.float32)
  return src_code + linspace * (dst_code - src_code)

print('Please upload the source image or \
use the default image by clicking `Cancel upload` button.')
uploaded = files.upload()
if uploaded:
  src_image_name = list(uploaded.keys())[0]
  src_image = align(inverter, src_image_name)
  if src_image.shape[2] == 4:
    src_image = src_image[:, :, :3]
  os.remove(src_image_name)
else:
  src_image_name = '000008.png'
  im_name = os.path.join(pre, src_image_name)
  src_image = align(inverter, im_name)
print('Source image ready!!!')
src_code_path = os.path.join(inverted_code_dir, 
                             src_image_name.split('.')[0] + '.npy')

print('Please upload the target image or \
use the default image by clicking `Cancel upload` button.')
uploaded = files.upload()
if uploaded:
  dst_image_name = list(uploaded.keys())[0]
  dst_image = align(inverter, dst_image_name)
  if dst_image.shape[2] == 4:
    dst_image = dst_image[:, :, :3]
  os.remove(dst_image_name)
else:
  dst_image_name = '000013.png'
  im_name = os.path.join(pre, dst_image_name)
  dst_image = align(inverter, im_name)
print('Target image ready!!!')
sys.stdout.flush()
dst_code_path = os.path.join(inverted_code_dir, 
                             dst_image_name.split('.')[0] + '.npy')

if not os.path.exists(src_code_path):
  src_code, _ = invert(inverter, src_image)
  np.save(src_code_path, src_code)
else:
  src_code = np.load(src_code_path)


if not os.path.exists(dst_code_path):
  dst_code, _ = invert(inverter, dst_image)
  np.save(dst_code_path, dst_code)
else:
  dst_code = np.load(dst_code_path)
print()
print('Both the source image and target image are inverted, \
please use the next block to interpolate!!!')

In [ ]:
#@title { display-mode: "form", run: "auto" }
step = 2 #@param {type:"slider", min:1, max:10, step:1}

inter_images = []
inter_images.insert(0, dst_image)
inter_images.insert(-1, src_image)

inter_codes = linear_interpolate(np.reshape(src_code, [1, -1]),
                                 np.reshape(dst_code, [1, -1]),
                                 step=step)
inter_codes = np.reshape(inter_codes, [-1, inverter.G.num_layers, inverter.G.w_space_dim])
inter_imgs = generator.easy_synthesize(inter_codes, **{'latent_space_type': 'wp'})['image']

for ind in range(inter_imgs.shape[0]):
  inter_images.insert(ind+1, inter_imgs[ind])

inter_images = np.asarray(inter_images)
imshow(inter_images, col=inter_images.shape[0])

## Caffe model, function to plot results from its results

In [ ]:
# Function used to plot results from Caffe model

import numpy as np
import matplotlib.pyplot as plt

def plot_caffe():
    labels = ['-4', '-3', '-2', '-1', '1', '2', '3', '4']

    attrs_cum = np.zeros((8, 40))
    with open('./result/demo_result_int.list') as f:
        for idx,line in enumerate(f):
            attrs = line.split()[1:]
            if idx%9 == 0:
                base_attrs = attrs
                title = line.split('/')[4]
            else:
                for idx2,attr in enumerate(attrs):
                    if attr != base_attrs[idx2]:
                        attrs_cum[(idx%9)-1, idx2] += 1
            if idx%117 == 116:
                fig,ax = plt.subplots(1)
                [a,b,c,d,e,f,g,h] = ax.plot(attrs_cum.T/13, label=labels);
                ax.legend([a,b,c,d,e,f,g,h], labels, loc=1)
                ax.set_xticks(np.linspace(0,40,41), minor=True)
                ax.grid(True, linestyle='--', linewidth=1, which='major')
                ax.grid(True, linestyle='--', linewidth=1, which='minor', alpha=0.5)
                plt.title(title)
                fig.show()
                attrs_cum = np.zeros((8, 40))

    features = np.array(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald',
    'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry',
    'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee',
    'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
    'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose',
    'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair',
    'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
    'Wearing_Necklace', 'Wearing_Necktie', 'Young'])

    print(np.where(features == 'Smiling'))
